In [90]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
# 使用headless无界面浏览器模式
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

In [91]:
def write_one_page(writer,soup):
    for num in range(1,5):
        table=soup.find('table',attrs={'id':'listTable'+str(num)})
        rows=table.find_all('tr')
        for row in rows:
            record=[]
            cols=row.find_all('td')
            for col in cols:
                record.append(col.get_text().strip())
            if len(record)==0:
                continue
            else:
                writer.writerow(record)

In [94]:
def permno_craw(permno,market_type,driver):
    url='http://quote.eastmoney.com/f1.html?code={}&market={}'
    driver.get(url.format(permno,market_type))
    soup = BeautifulSoup(driver.page_source, "html.parser")
    counts=soup.find('span',attrs={'class':'count'}).get_text().strip()
    if int(counts)==0:
        return [permno,'No data',0]
    else:
        data_file= '/Users/zhangdi/Desktop/ceshi/{}.csv'.format(permno)
        with open(data_file,'w',newline='',encoding='utf8') as wf:
            writer = csv.writer(wf)
            head_csv=['time','price','volumn']
            writer.writerow(head_csv)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            write_one_page(writer=writer,soup=soup)
            for page_number in range(2,(int(counts)+1)):
                driver.find_element_by_css_selector('[class="go"]').send_keys(str(page_number))
                driver.find_element_by_css_selector('[class="goBtn fl"]').click()
                soup = BeautifulSoup(driver.page_source, "html.parser")
                write_one_page(writer=writer,soup=soup)
            return[permno,'finished',counts]

In [95]:
stocks={'603799':'1',
        '002156':'2',
        '002463':'2',
        '000001':'2'}
results=dict()
driver = webdriver.Chrome(chrome_options=chrome_options)
for stock,market_type in stocks.items():
    result=permno_craw(stock,market_type,driver)
    results[stock]=result
    time.sleep(1)

/Users/zhangdi/Desktop/A_share/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [96]:
results

{'603799': ['603799', 'finished', '34'],
 '002156': ['002156', 'finished', '33'],
 '002463': ['002463', 'finished', '33'],
 '000001': ['000001', 'finished', '33']}